In [ ]:
import numpy as np
from Orange.data import Domain, ContinuousVariable, Table

# Input data from Orange
data = in_data
domain = data.domain

# Per-36 stat columns in your dataset
stat_names = [
    "pts_per_36_min",
    "trb_per_36_min",
    "ast_per_36_min",
    "stl_per_36_min",
    "blk_per_36_min",
    "tov_per_36_min",
    "pf_per_36_min",
]

# Safely get a column as a numpy array (zeros if missing)
def get_col(name):
    if name in domain:
        var = domain[name]
        # New recommended way: get_column
        return data.get_column(var).astype(float)
    else:
        return np.zeros(len(data), dtype=float)

pts, trb, ast, stl, blk, tov, pf = [get_col(n) for n in stat_names]

# Raw impact score (per 36 minutes)
impact_raw = pts + trb + ast + stl + blk - tov - pf

# Compute z-score of impact
mean = np.nanmean(impact_raw)
std = np.nanstd(impact_raw)

if std == 0 or np.isnan(std):
    impact_z = np.zeros_like(impact_raw)
else:
    impact_z = (impact_raw - mean) / std

# Create new variable and extend the domain
impact_var = ContinuousVariable("impact_z")
new_domain = Domain(
    domain.attributes + (impact_var,),
    domain.class_vars,
    domain.metas
)

# Add the new column to X
new_X = np.hstack((data.X, impact_z.reshape(-1, 1)))

# Build output table
out_data = Table(new_domain, new_X, data.Y, data.metas)

In [ ]:
import numpy as np
from Orange.data import Domain, ContinuousVariable, Table
from sklearn.neighbors import LocalOutlierFactor

# Input from Orange
data = in_data

# Numeric matrix (attributes only)
X = np.array(data.X, dtype=float)

# ---- Handle missing values (NaNs) by mean imputation ----
# Compute column means ignoring NaNs
col_means = np.nanmean(X, axis=0)

# Find indices where NaN appears and replace with column means
inds = np.where(np.isnan(X))
X[inds] = col_means[inds[1]]

# ---- Compute LOF with same settings as your Orange LOF widget ----
lof = LocalOutlierFactor(
    n_neighbors=10,
    contamination=0.10,
    metric='manhattan'
)

_ = lof.fit_predict(X)
lof_raw = -lof.negative_outlier_factor_   # higher = more outlier

# Normalize LOF scores to 0–1 for nicer visualization
lof_min = np.min(lof_raw)
lof_max = np.max(lof_raw)
if lof_max == lof_min:
    lof_score = np.zeros_like(lof_raw)
else:
    lof_score = (lof_raw - lof_min) / (lof_max - lof_min)

# ---- Add LOF_score as a new continuous attribute ----
lof_var = ContinuousVariable("LOF_score")

new_domain = Domain(
    data.domain.attributes + (lof_var,),
    data.domain.class_vars,
    data.domain.metas
)

new_X = np.hstack([data.X, lof_score.reshape(-1, 1)])

out_data = Table(new_domain, new_X, data.Y, data.metas)